In [ ]:
import torch
import torch.nn as nn
from torchvision.models.vgg import vgg16

device = "cuda" if torch.cuda.is_available() else "cpu"

model = vgg16( weights= 'VGG16_Weights.DEFAULT')
model.classifier[6] = nn.Linear(in_features=4096, out_features=10, bias=True)
model

In [ ]:
model.to(device)

In [9]:
import torch
from torchvision.datasets.cifar import CIFAR10
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_dataset = CIFAR10(root = './', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root = './', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset,batch_size = 32, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size = 32, shuffle=False)

100%|██████████| 170M/170M [04:56<00:00, 576kB/s] 


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [ ]:
from torch.optim import Adam
from tqdm import tqdm
lr = 1e-3
optim = Adam(model.parameters(), lr=lr)
epochs = 1
# 학습루프
for epoch in range(epochs):
    tqdm_obj = tqdm(train_loader,desc=f'epoch : {epoch+1}/{epochs}')
    for data, label in tqdm_obj:
        optim.zero_grad()
        preds = model(data.to(device))
        loss = nn.CrossEntropyLoss()(preds, label.to(device))
        loss.backward()
        optim.step()

        tqdm_obj.set_postfix(loss=loss.item())
        

torch.save(model.state_dict(), 'vgg16_custom.pth')  